In [1]:
import pandas as pd
import os

In [2]:
DERS_LISTESI_PATH = r'Ders Listesi.xlsx'
OGRENCI_LISTESI_PATH = r'ogrenci_listesi.xlsx'

In [3]:
import re

def oku_ders_listesi(path: str, sheet_name: str = "Ders Listesi") -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name, header=None)

    sinif_rows = df[df[0].astype(str).str.contains("Sınıf", case=False, na=False)].reset_index()

    rows = []
    for i in range(len(sinif_rows)):
        start = sinif_rows.loc[i, "index"] + 2 
        end = sinif_rows.loc[i+1, "index"] if i+1 < len(sinif_rows) else len(df)
        sinif_text = str(df.loc[sinif_rows.loc[i, "index"], 0])

        sinif_match = re.search(r'(\d+)', sinif_text)
        sinif = int(sinif_match.group(1)) if sinif_match else 0

        block = df.iloc[start:end, :3].copy()
        block.columns = ["DERS KODU", "DERSİN ADI", "DERSİ VEREN ÖĞR. ELEMANI"]
        secmeli_mask = block["DERS KODU"].astype(str).str.contains("Seçimlik|Seçmeli", case=False, na=False)

        if secmeli_mask.any():
            secmeli_index = secmeli_mask.idxmax()
            block = block.drop(secmeli_index)
            
            block["Seçmeli mi?"] = False
            block.loc[secmeli_index+1:, "Seçmeli mi?"] = True
        else:
            block["Seçmeli mi?"] = False

        block["SINIF"] = sinif
        rows.append(block)

    final_df = pd.concat(rows, ignore_index=True)
    
    final_df = final_df[~final_df["DERS KODU"].isin(['DERS KODU', 'DERSİN ADI', 'DERSİ VEREN ÖĞR. ELEMANI'])]
    
    final_df.to_excel("processed_ders_listesi.xlsx", index=False)
    
    return final_df

In [ ]:
import os
import pandas as pd
from typing import List
import re

def get_year_from_str(year_str: str) -> int:
    sinif_match = re.search(r'(\d+)', year_str) 
    if sinif_match:
        return int(sinif_match.group(1))
    else:
        raise ValueError(f"year string '{year_str}' does not contain a valid year")

def process_class_list(df: pd.DataFrame, department, strict: bool = True) -> pd.DataFrame:

    sinif_rows = df[df[0].astype(str).str.contains("Sınıf", case=False, na=False)].reset_index()

    rows: List[pd.DataFrame] = []
    for i in range(len(sinif_rows)):
        start = sinif_rows.loc[i, "index"] + 2
        end = sinif_rows.loc[i + 1, "index"] if i + 1 < len(sinif_rows) else len(df)
        sinif_text = str(df.loc[sinif_rows.loc[i, "index"], 0])
        sinif = get_year_from_str(sinif_text)

        try:
            block = df.iloc[start:end, :3].copy()
            if block.shape[1] < 3:
                msg = (f"[HATA] {i}. sınıf bloğunda beklenen 3 sütun yok. "
                       f"Bulunan sütun sayısı: {block.shape[1]} | Aralık: [{start}, {end})")
                print(msg)
                if strict:
                    return {
                        'df': df,
                        'status': 'error',
                        'message': msg
                    }
                else:
                    continue

            block.columns = ["class_id", "class_name", "teacher"]

            for ridx, r in block.iterrows():
                for col in ["class_id", "class_name", "teacher"]:
                    try:
                        val = str(r[col])
                        if val.strip().lower() in ["nan", "none", ""]:
                            raise ValueError("Boş değer")
                    except Exception as e_row:
                        excel_row_no = int(ridx) + 1
                        print(f"[HATA] Satır {excel_row_no}, sütun '{col}' işlenemedi.")
                        print(f"  - Sınıf: {sinif}")
                        print(f"  - Hata Türü: {type(e_row).__name__} | Mesaj: {e_row}")
                        print(f"  - Orijinal değer: {r.get(col, None)}\n")
                        block.at[ridx, col] = None  # hücreyi None olarak işaretle
                        if strict:
                            return {
                                'df': df,
                                'status': 'error',
                                'message': f"Satır {excel_row_no}, sütun '{col}' işlenemedi: {e_row}"
                            }

            secmeli_mask = block["class_id"].astype(str).str.contains("Seçimlik|Seçmeli", case=False, na=False)
            if secmeli_mask.any():
                secmeli_index = secmeli_mask.idxmax()
                block = block.drop(secmeli_index)
                block["is_optional"] = False
                block.loc[block.index >= secmeli_index, "is_optional"] = True
            else:
                block["is_optional"] = False

            block["grade"] = sinif
            block = block[~block["class_id"].isin(['class_id', 'class_name', 'teacher'])]

            rows.append(block)

        except Exception as e:
            print("\n[HATA] Bir sınıf bloğu işlenemedi:")
            print(f"  - Sınıf başlığı: '{sinif_text}' (çıkarılan yıl: {sinif})")
            print(f"  - Aralık: [{start}, {end})")
            print(f"  - Hata Türü: {type(e).__name__} | Mesaj: {e}\n")
            if strict:
                return {
                    'df': df,
                    'status': 'error',
                    'message': f"Sınıf bloğu işlenemedi: '{sinif_text}' | Hata: {e}"
                }
            else:
                continue

    if not rows:
        msg = "[HATA] Hiçbir blok üretilemedi. Girdi formatını kontrol edin."
        print(msg)
        if strict:
            raise ValueError(msg)
        return pd.DataFrame(columns=["DERS KODU", "DERSİN ADI", "DERSİ VEREN ÖĞR. ELEMANI", "Seçmeli mi?", "SINIF"])

    final_df = pd.concat(rows, ignore_index=True)
    final_df["department"] = department if department is not None else "Unknown"

    return {
        'df':final_df,
        'status':'success',
        'message':'Class list processed successfully.'
    }

df = pd.read_excel(DERS_LISTESI_PATH, sheet_name='Ders Listesi', header=None)
return_dict = process_class_list(df, department="Computer Science")
final_df = return_dict.get('df', df)
final_df.to_excel("final_processed_ders_listesi.xlsx", index=False)

[HATA] Satır 29, sütun 'class_name' işlenemedi.
  - Sınıf: 3
  - Hata Türü: ValueError | Mesaj: Boş değer
  - Orijinal değer: nan



In [4]:
ders_list_df = oku_ders_listesi(DERS_LISTESI_PATH)
print(ders_list_df)

   DERS KODU                                     DERSİN ADI  \
0     AIT109           Atatürk İlkeleri ve İnkılap Tarihi I   
1     TDB107                                    Türk Dili I   
2     YDB117                                    İngilizce I   
3     FEF111                                        Fizik I   
4     FEF113                                   Lineer Cebir   
5     FEF115                                    Matematik I   
6     BLM101                      Bilgisayar Laboratuvarı I   
7     BLM103                Bilgisayar Mühendisliğine Giriş   
8     BLM105                                 Programlama I    
9     FEF203                        Diferansiyel Denklemler   
10    MUH201                    Nesneye Yönelik Programlama   
11    BLM207                 Veri Yapıları ve Algoritmaları   
12    BLM209                  Programlama Laboratuvarı - I    
13    BLM211               Mantıksal Tasarım ve Uygulamalar   
14    BLM205                                Ayrık Matem

In [5]:
def read_data(DATA_PATH):
    if DATA_PATH.endswith('.csv'):
        return pd.read_csv(DATA_PATH)
    elif DATA_PATH.endswith('.xlsx'):
        return pd.read_excel(DATA_PATH)
    else:
        raise ValueError("Unsupported file format")

In [6]:
ögrenci_list_df = read_data(OGRENCI_LISTESI_PATH)
print(ögrenci_list_df)

      Öğrenci No       Ad Soyad     Sınıf    Ders
0      210059017  Cansu Bozkurt  5. Sınıf  BLM401
1      210059017  Cansu Bozkurt  5. Sınıf  BLM405
2      210059017  Cansu Bozkurt  5. Sınıf  BLM411
3      210059017  Cansu Bozkurt  5. Sınıf  BLM417
4      210059017  Cansu Bozkurt  5. Sınıf  BLM421
...          ...            ...       ...     ...
3895   259757258     Nil Öztürk  1. Sınıf  FEF111
3896   259757258     Nil Öztürk  1. Sınıf  FEF113
3897   259757258     Nil Öztürk  1. Sınıf  FEF115
3898   259757258     Nil Öztürk  1. Sınıf  TDB107
3899   259757258     Nil Öztürk  1. Sınıf  YDB117

[3900 rows x 4 columns]


In [7]:
ögrenci_list_df

,Öğrenci No,Ad Soyad,Sınıf,Ders
0,210059017,Cansu Bozkurt,5. Sınıf,BLM401
1,210059017,Cansu Bozkurt,5. Sınıf,BLM405
2,210059017,Cansu Bozkurt,5. Sınıf,BLM411
3,210059017,Cansu Bozkurt,5. Sınıf,BLM417
4,210059017,Cansu Bozkurt,5. Sınıf,BLM421
...,...,...,...,...
3895,259757258,Nil Öztürk,1. Sınıf,FEF111
3896,259757258,Nil Öztürk,1. Sınıf,FEF113
3897,259757258,Nil Öztürk,1. Sınıf,FEF115
3898,259757258,Nil Öztürk,1. Sınıf,TDB107
